# Regression and Other Stories: Gay

In [2]:
import arviz as az
from bambi import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [5]:
data = pd.read_csv("https://raw.githubusercontent.com/avehtari/ROS-Examples/master/Gay/data/naes04.csv")
data.head()

,Unnamed: 0,age,gender,race,gayFavorFederalMarriage,gayFavorStateMarriage,gayKnowSomeone
0,1,70.0,Female,Hispanic,No,NaN,NaN
1,2,54.0,Female,White,No,NaN,NaN
2,3,74.0,Male,White,No,NaN,NaN
3,4,73.0,Female,Other,Yes,No,No
4,5,48.0,Female,White,No,Yes,Yes


In [ ]:
# TODO: Rest of notebook